In [ ]:
# Packages
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.set_option('display.max_colwidth', 100)

In [ ]:
train_data = pd.read_csv('./train.csv')
test_data = pd.read_csv('./test.csv')
solution =  pd.read_csv('./solution_template.csv')

In [ ]:
# Drop constant columns, or with heavy missingness gender is constant and first_novel_treatment and first_novel_treatment_type has lots of missing values
df_train = train_data.drop(['metastatic_first_novel_treatment', 'metastatic_first_novel_treatment_type', 'patient_gender'], axis=1)
df_test = test_data.drop(['metastatic_first_novel_treatment', 'metastatic_first_novel_treatment_type', 'patient_gender'], axis=1)

#combining both data sets
df_test['metastatic_diagnosis_period'] = -1
df = pd.concat([df_train, df_test], axis=0, ignore_index=True)
df['source'] = np.where(df['metastatic_diagnosis_period'] == -1, 'test', 'train')

In [ ]:
df[['patient_state','patient_zip3','source']].drop_duplicates().groupby(['source','patient_zip3'], as_index=False)['patient_state']. \
    count().sort_values('patient_state', ascending=False).reset_index(drop=True).head()

,source,patient_zip3,patient_state
0,train,630,2
1,train,864,2
2,test,100,1
3,train,404,1
4,train,412,1


In [ ]:
df['patient_state'] = np.where(df['patient_zip3'] == 630, 'MO', np.where(df['patient_zip3'] == 864, 'AZ', df['patient_state']))

In [ ]:
df['Division'] = np.where(df['patient_state'] == 'MO', 'West North Central', df['patient_state'])

In [ ]:
pop_cols = df.loc[:, 'population':'veteran'].columns.to_list()
df_pop = df[['patient_zip3', 'patient_state'] + pop_cols].drop_duplicates().sort_values('patient_zip3')
print(df_pop.shape)
df_pop.head()

(766, 66)


,patient_zip3,patient_state,population,density,age_median,age_under_10,age_10_to_19,age_20s,age_30s,age_40s,age_50s,age_60s,age_70s,age_over_80,male,female,married,divorced,never_married,widowed,family_size,family_dual_income,income_household_median,income_household_under_5,income_household_5_to_10,income_household_10_to_15,income_household_15_to_20,income_household_20_to_25,income_household_25_to_35,income_household_35_to_50,income_household_50_to_75,income_household_75_to_100,income_household_100_to_150,income_household_150_over,income_household_six_figure,income_individual_median,home_ownership,housing_units,home_value,rent_median,rent_burden,education_less_highschool,education_highschool,education_some_college,education_bachelors,education_graduate,education_college_or_above,education_stem_degree,labor_force_participation,unemployment_rate,self_employed,farmer,race_white,race_black,race_asian,race_native,race_pacific,race_other,race_multiple,hispanic,disabled,poverty,limited_english,commute_time,health_uninsured,veteran
69,100,NY,37662.55,29851.69,37.90,8.70,7.81,17.88,20.12,13.02,11.84,9.78,6.60,4.20,48.10,51.90,36.08,10.40,49.31,4.20,3.00,53.72,113117.00,4.41,3.26,4.57,3.19,2.76,5.27,6.73,10.30,8.91,13.80,36.81,50.60,69170.29,24.20,17281.81,1025205.0,2175.51,28.84,9.87,11.37,12.34,34.36,32.05,66.41,43.38,68.81,6.60,18.53,0.02,54.97,13.83,14.02,0.33,0.05,10.01,6.81,21.14,9.11,14.42,8.29,31.40,4.43,2.08
1424,101,NY,2746.14,5363.17,41.25,13.85,6.75,10.80,16.95,15.40,8.50,10.85,8.75,8.15,42.55,57.45,46.35,10.40,35.45,7.85,2.85,46.40,113609.50,0.75,1.90,1.35,7.75,1.10,5.35,1.70,4.95,24.80,14.85,35.50,50.35,88910.50,15.85,1389.05,1178137.0,2701.50,31.90,5.00,16.65,7.20,38.35,32.75,71.10,38.35,68.80,2.50,20.86,0.02,85.25,2.55,6.40,0.05,0.00,2.50,3.30,15.65,19.35,4.15,1.15,27.85,5.40,1.20
1179,103,NY,37938.00,3697.29,39.87,11.61,13.14,12.73,12.78,13.23,14.12,12.38,6.58,3.44,49.16,50.84,49.43,9.87,35.00,5.70,3.46,55.22,90381.92,4.13,2.24,3.58,2.77,2.46,5.69,8.14,14.72,10.74,19.09,26.47,45.56,42708.58,67.34,13040.62,568177.2,1463.75,34.94,11.95,29.73,24.02,20.72,13.61,34.33,44.51,59.85,5.23,12.96,0.00,67.88,12.19,8.69,0.36,0.03,4.73,6.10,20.46,9.92,10.93,6.44,44.79,4.58,3.28
14,104,NY,58934.16,16936.58,36.22,13.46,13.31,15.06,13.67,12.46,12.61,9.60,6.04,3.75,47.38,52.62,32.32,14.19,47.87,5.62,3.39,45.59,50831.96,6.02,5.56,8.63,5.86,5.42,9.28,12.16,15.19,9.81,11.57,10.51,22.08,27712.36,23.28,21012.52,422924.6,1317.64,41.36,24.51,27.48,25.06,14.24,8.67,22.90,42.65,58.96,10.86,16.05,0.00,24.15,32.52,3.52,0.72,0.12,30.26,8.70,53.74,16.10,25.03,15.92,44.94,7.47,2.52
1506,105,NY,10690.73,910.85,43.49,9.90,15.26,10.20,10.85,12.72,15.35,13.88,7.40,4.43,51.06,48.94,53.56,8.99,32.97,4.49,3.21,61.77,147128.20,1.97,0.98,1.50,1.62,2.05,3.91,5.84,9.75,9.39,17.06,45.95,63.01,57872.20,75.79,3801.95,620196.9,2061.17,32.79,7.30,15.13,18.84,28.79,29.95,58.74,43.14,64.75,7.06,17.37,0.00,72.17,7.65,6.96,0.17,0.01,8.13,4.90,15.90,9.39,7.85,3.28,36.81,3.62,3.97


In [ ]:
df_pop.groupby('patient_zip3', as_index=False)['population'].count().sort_values('population', ascending=False).reset_index(drop=True).head()

,patient_zip3,population
0,988,2
1,395,2
2,968,2
3,100,1
4,675,1


In [ ]:
df_pop.loc[df_pop.patient_zip3.isin([988, 395, 968])]
df.drop(columns=['bmi'], inplace=True)

In [ ]:
# inconsistencies for patient_zip3==395
df.loc[df.patient_id == 714510, pop_cols] = df.loc[df.patient_id == 636245, pop_cols].values

# inconsistencies for patient_zip3==968
df.loc[df.patient_id == 271422, pop_cols] = df.loc[df.patient_id == 271245, pop_cols].values

# inconsistencies for patient_zip3==988
df.loc[df.patient_id == 441322, pop_cols] = df.loc[df.patient_id == 982003, pop_cols].values

# Also fix in population subset
df_pop.loc[df_pop.index==2414,:] = df_pop.loc[df_pop.index==1670,:].values
df_pop.loc[df_pop.index==1847,:] = df_pop.loc[df_pop.index==31,:].values
df_pop.loc[df_pop.index==10245,:] = df_pop.loc[df_pop.index==422,:].values
df_pop = df_pop.drop_duplicates()

# Check if duplicates remain
df[['patient_zip3'] + pop_cols].drop_duplicates().shape

(763, 65)

In [ ]:
#here we can observe that there are various breast diagnosis code that belong to men breast cancer but the data given to us contains only women so lets fix that
# also C509 is the only breast cancer code with 3 digits its same as C5091 so lets make it that

df['breast_cancer_diagnosis_code'] = df['breast_cancer_diagnosis_code'].replace({
    # Recode male to female
    'C50122': 'C50112', 'C50221': 'C50211', 'C50421': 'C50411', 'C50922': 'C50912',
    # Recode categories
    'C509': 'C5091'
})

In [ ]:
df_pop[(df_pop.patient_state == 'TX') & (~df_pop.limited_english.isna())].head()

,patient_zip3,patient_state,population,density,age_median,age_under_10,age_10_to_19,age_20s,age_30s,age_40s,age_50s,age_60s,age_70s,age_over_80,male,female,married,divorced,never_married,widowed,family_size,family_dual_income,income_household_median,income_household_under_5,income_household_5_to_10,income_household_10_to_15,income_household_15_to_20,income_household_20_to_25,income_household_25_to_35,income_household_35_to_50,income_household_50_to_75,income_household_75_to_100,income_household_100_to_150,income_household_150_over,income_household_six_figure,income_individual_median,home_ownership,housing_units,home_value,rent_median,rent_burden,education_less_highschool,education_highschool,education_some_college,education_bachelors,education_graduate,education_college_or_above,education_stem_degree,labor_force_participation,unemployment_rate,self_employed,farmer,race_white,race_black,race_asian,race_native,race_pacific,race_other,race_multiple,hispanic,disabled,poverty,limited_english,commute_time,health_uninsured,veteran
113,750,TX,41306.45,1108.94,36.90,13.35,14.67,12.56,14.57,14.90,13.60,9.14,5.06,2.19,49.65,50.35,56.08,11.07,29.15,3.69,3.31,60.91,96428.52,1.75,1.23,2.03,2.07,2.43,5.83,9.68,15.57,13.57,19.76,26.03,45.80,46804.11,63.78,14545.66,304967.3,1478.24,28.18,10.07,18.21,27.56,28.32,15.86,44.18,45.51,70.67,4.16,14.99,0.78,61.69,11.25,13.56,0.54,0.05,4.57,8.33,23.30,8.16,7.87,5.72,27.79,13.64,5.92
193,751,TX,16981.35,271.91,38.54,13.25,15.18,11.81,11.80,12.72,14.05,11.84,6.37,2.96,49.87,50.13,52.83,13.57,28.02,5.57,3.41,52.71,68254.40,2.77,1.97,3.45,3.57,4.53,8.93,13.51,19.20,13.47,16.68,11.94,28.63,32549.37,75.95,5699.46,170833.6,1113.77,30.60,17.10,31.84,32.40,12.76,5.89,18.65,38.15,62.16,5.67,13.56,4.60,69.76,15.89,1.13,0.66,0.09,4.86,7.59,25.48,14.05,11.85,3.36,34.21,17.66,7.20
217,752,TX,27504.46,1774.32,34.47,12.47,11.93,17.80,16.93,12.70,11.80,9.28,4.67,2.43,51.18,48.82,38.35,13.59,43.80,4.24,3.25,53.95,66939.67,4.31,2.93,4.33,4.19,4.68,8.62,13.24,17.82,11.30,12.89,15.72,28.60,38888.60,40.59,10797.28,330004.9,1250.98,31.87,18.95,20.41,22.41,23.48,14.76,38.24,42.07,68.39,5.78,16.17,0.29,55.78,23.76,3.73,0.48,0.03,8.23,8.01,36.19,11.05,17.46,8.92,26.98,20.98,4.76
387,754,TX,5140.34,31.83,42.46,11.46,14.03,10.52,11.22,11.95,14.14,13.26,9.49,3.95,50.03,49.97,55.95,12.91,22.90,8.23,3.09,48.39,63523.28,3.41,2.39,5.47,3.93,4.34,9.70,11.76,18.44,13.70,16.34,10.54,26.88,32237.50,77.92,1862.55,154794.3,930.58,46.56,14.12,33.95,32.89,13.57,5.47,19.05,40.41,58.53,4.30,13.62,12.28,85.63,4.86,0.66,0.79,0.12,2.88,5.07,12.74,17.28,13.18,1.30,29.59,15.86,8.78
594,755,TX,5235.38,48.73,42.13,12.87,13.54,11.09,11.65,12.09,11.67,14.73,8.33,4.02,50.98,49.03,52.34,14.09,25.92,7.65,3.25,45.30,52211.19,3.15,3.03,5.20,9.05,4.37,10.06,14.38,19.20,13.95,11.53,6.06,17.58,28585.77,73.38,1955.46,101723.7,724.58,26.87,11.53,45.04,28.02,9.80,5.62,15.42,41.40,53.34,5.67,11.40,8.65,80.80,13.98,0.65,0.34,0.01,2.08,2.14,8.45,16.09,17.21,0.72,25.28,15.68,8.60


In [ ]:
#here we are going to fill the data with state Mean if the data is numerical , with mode if otherwise
def mixed_imputation(df, group_col):
    for column in df.columns:
        if column != group_col:  # Exclude the group column
            # If the column is numerical, then mean imputation
            if df[column].dtype in [np.dtype('float_'), np.dtype('int_')]:
                mean_impute = df.groupby(group_col)[column].mean()
                df[column] = df[column].fillna(df[group_col].map(mean_impute))

            # If the column is categorical, apply mode imputation
            else :
                mode_impute = df.groupby(group_col)[column].apply(lambda x: x.mode()[0] if not x.mode().empty else np.nan)
                df[column] = df[column].fillna(df[group_col].map(mode_impute))

    return df

# Impute missing values
df_pop = mixed_imputation(df=df_pop, group_col='patient_state')

In [ ]:
temp_cols = df.columns[df.columns.str.startswith('Average')].tolist()
df_temp = df[['patient_zip3', 'patient_state'] + temp_cols].drop_duplicates().sort_values(by='patient_zip3').reset_index(drop=True)

print(df_temp.shape)
df_temp.head()

(763, 74)


,patient_zip3,patient_state,Average of Jan-13,Average of Feb-13,Average of Mar-13,Average of Apr-13,Average of May-13,Average of Jun-13,Average of Jul-13,Average of Aug-13,Average of Sep-13,Average of Oct-13,Average of Nov-13,Average of Dec-13,Average of Jan-14,Average of Feb-14,Average of Mar-14,Average of Apr-14,Average of May-14,Average of Jun-14,Average of Jul-14,Average of Aug-14,Average of Sep-14,Average of Oct-14,Average of Nov-14,Average of Dec-14,Average of Jan-15,Average of Feb-15,Average of Mar-15,Average of Apr-15,Average of May-15,Average of Jun-15,Average of Jul-15,Average of Aug-15,Average of Sep-15,Average of Oct-15,Average of Nov-15,Average of Dec-15,Average of Jan-16,Average of Feb-16,Average of Mar-16,Average of Apr-16,Average of May-16,Average of Jun-16,Average of Jul-16,Average of Aug-16,Average of Sep-16,Average of Oct-16,Average of Nov-16,Average of Dec-16,Average of Jan-17,Average of Feb-17,Average of Mar-17,Average of Apr-17,Average of May-17,Average of Jun-17,Average of Jul-17,Average of Aug-17,Average of Sep-17,Average of Oct-17,Average of Nov-17,Average of Dec-17,Average of Jan-18,Average of Feb-18,Average of Mar-18,Average of Apr-18,Average of May-18,Average of Jun-18,Average of Jul-18,Average of Aug-18,Average of Sep-18,Average of Oct-18,Average of Nov-18,Average of Dec-18
0,100,NY,33.44,30.84,38.45,51.26,60.18,71.91,78.55,71.32,64.49,56.13,41.99,34.79,24.37,26.97,33.92,49.99,61.76,70.54,74.73,71.48,66.73,55.69,41.66,38.25,26.53,20.84,34.33,50.51,66.72,69.80,75.96,75.25,71.07,54.58,48.87,46.33,30.06,35.06,46.52,51.23,60.48,72.47,77.11,77.54,70.08,56.80,45.79,35.42,35.37,37.89,37.02,56.56,59.65,71.17,76.11,72.17,69.26,60.80,42.89,32.62,27.67,37.57,37.87,47.55,65.06,70.23,77.12,77.72,69.82,55.79,42.04,37.18
1,101,NY,33.51,30.81,38.44,51.27,60.14,71.98,78.64,71.34,64.53,56.15,42.03,34.93,24.42,26.97,33.92,NaN,61.76,NaN,74.75,71.51,66.77,55.65,41.72,38.32,26.62,20.97,34.39,50.56,66.82,69.84,76.02,75.35,71.20,54.72,48.96,46.43,30.09,35.16,46.64,51.34,60.55,72.67,77.19,77.67,70.22,56.95,45.83,35.54,35.45,37.99,37.09,56.75,59.79,71.37,76.24,72.29,69.38,60.93,43.01,32.70,27.77,37.69,37.95,47.68,65.17,70.33,77.22,77.83,69.93,55.92,42.16,37.26
2,103,NY,34.01,31.87,38.52,51.63,60.29,71.92,78.43,71.46,64.39,56.21,42.56,35.83,25.57,26.97,34.67,51.46,61.74,70.47,74.70,71.37,66.91,56.54,41.69,38.68,27.28,21.76,35.17,50.75,66.20,69.40,75.87,75.20,70.58,54.21,48.93,47.42,30.88,35.79,46.99,51.55,60.28,71.85,77.05,77.30,70.17,56.61,46.67,36.33,36.01,39.22,37.09,56.44,59.80,70.75,76.06,72.15,69.15,60.29,43.77,32.91,28.04,38.68,38.14,47.99,65.17,70.81,76.90,77.75,69.90,56.45,42.55,37.49
3,104,NY,32.81,31.04,38.55,51.15,60.48,71.39,77.86,71.19,64.10,55.98,41.73,33.68,23.95,26.97,33.92,49.99,61.76,70.54,74.55,71.22,66.44,56.05,41.21,37.70,25.83,19.78,33.79,50.08,65.92,69.54,75.48,74.45,70.01,53.43,48.15,45.55,29.80,34.24,45.55,50.29,59.92,70.84,76.48,76.47,68.93,55.55,45.48,34.50,34.72,37.03,36.50,55.00,58.52,69.56,75.05,71.20,68.33,59.81,41.91,31.91,26.81,36.58,37.22,46.54,64.18,69.41,76.37,76.81,68.91,54.75,41.06,36.54
4,105,NY,29.13,27.78,34.54,48.12,58.59,67.86,75.06,68.19,61.44,53.31,38.38,31.92,20.03,20.35,28.98,46.86,59.60,67.73,72.00,68.59,64.27,54.66,38.04,34.64,22.47,15.58,30.56,47.75,63.83,66.95,72.20,71.25,67.61,50.64,45.29,42.99,26.84,31.00,42.83,47.79,57.43,67.18,73.73,73.14,66.10,52.98,43.58,32.09,31.65,34.71,33.61,52.17,56.50,66.72,71.75,68.49,65.46,57.39,40.07,28.83,23.77,33.59,34.47,43.69,62.71,66.81,73.70,73.99,66.41,52.29,38.51,33.56


In [ ]:
df_temp.isnull().sum()

patient_zip3         0
patient_state        0
Average of Jan-13    2
Average of Feb-13    1
Average of Mar-13    0
Average of Apr-13    0
Average of May-13    1
Average of Jun-13    1
Average of Jul-13    0
Average of Aug-13    2
Average of Sep-13    2
Average of Oct-13    3
Average of Nov-13    1
Average of Dec-13    1
Average of Jan-14    1
Average of Feb-14    2
Average of Mar-14    2
Average of Apr-14    7
Average of May-14    0
Average of Jun-14    6
Average of Jul-14    0
Average of Aug-14    0
Average of Sep-14    0
Average of Oct-14    0
Average of Nov-14    2
Average of Dec-14    0
Average of Jan-15    1
Average of Feb-15    1
Average of Mar-15    1
Average of Apr-15    2
Average of May-15    0
Average of Jun-15    0
Average of Jul-15    0
Average of Aug-15    2
Average of Sep-15    0
Average of Oct-15    1
Average of Nov-15    1
Average of Dec-15    2
Average of Jan-16    1
Average of Feb-16    1
Average of Mar-16    0
Average of Apr-16    0
Average of May-16    2
Average of 

In [ ]:
df_temp[df_temp['Average of Aug-15'].isna()]

,patient_zip3,patient_state,Average of Jan-13,Average of Feb-13,Average of Mar-13,Average of Apr-13,Average of May-13,Average of Jun-13,Average of Jul-13,Average of Aug-13,Average of Sep-13,Average of Oct-13,Average of Nov-13,Average of Dec-13,Average of Jan-14,Average of Feb-14,Average of Mar-14,Average of Apr-14,Average of May-14,Average of Jun-14,Average of Jul-14,Average of Aug-14,Average of Sep-14,Average of Oct-14,Average of Nov-14,Average of Dec-14,Average of Jan-15,Average of Feb-15,Average of Mar-15,Average of Apr-15,Average of May-15,Average of Jun-15,Average of Jul-15,Average of Aug-15,Average of Sep-15,Average of Oct-15,Average of Nov-15,Average of Dec-15,Average of Jan-16,Average of Feb-16,Average of Mar-16,Average of Apr-16,Average of May-16,Average of Jun-16,Average of Jul-16,Average of Aug-16,Average of Sep-16,Average of Oct-16,Average of Nov-16,Average of Dec-16,Average of Jan-17,Average of Feb-17,Average of Mar-17,Average of Apr-17,Average of May-17,Average of Jun-17,Average of Jul-17,Average of Aug-17,Average of Sep-17,Average of Oct-17,Average of Nov-17,Average of Dec-17,Average of Jan-18,Average of Feb-18,Average of Mar-18,Average of Apr-18,Average of May-18,Average of Jun-18,Average of Jul-18,Average of Aug-18,Average of Sep-18,Average of Oct-18,Average of Nov-18,Average of Dec-18
490,651,MO,33.19,33.96,37.50,51.49,63.76,73.36,75.73,75.33,71.70,56.01,41.32,30.44,25.28,24.76,39.24,54.56,65.58,74.26,73.48,77.95,67.71,57.93,37.82,36.81,30.33,25.68,43.22,57.70,65.44,74.25,77.49,NaN,72.96,47.28,48.50,42.59,29.91,37.99,49.96,57.37,63.35,78.14,78.42,77.23,72.31,61.69,50.96,32.05,34.50,44.65,48.35,58.39,64.19,74.08,79.45,72.62,71.11,59.79,47.61,34.81,28.88,35.86,43.09,49.63,73.18,78.77,79.57,78.22,72.69,57.47,38.03,37.27
713,937,CA,44.23,49.17,58.57,65.17,71.11,78.78,84.34,79.61,74.78,63.05,55.30,44.85,50.41,54.28,60.38,64.79,72.31,78.20,84.05,81.38,78.30,69.02,56.58,50.92,46.47,55.73,63.03,64.74,68.98,81.36,82.35,NaN,77.49,69.73,50.36,45.21,49.84,54.21,58.20,65.72,71.97,80.50,82.27,80.99,75.57,66.52,56.45,47.60,47.24,53.74,57.73,63.40,71.14,80.34,84.52,84.43,77.32,NaN,56.55,45.73,50.16,50.34,55.26,64.33,69.57,76.49,84.80,80.38,74.89,65.43,55.02,NaN


In [ ]:
df_temp_melt = pd.melt(df_temp, id_vars=['patient_zip3', 'patient_state'])

# Extract month and convert it to datetime
df_temp_melt['month'] = df_temp_melt['variable'].apply(lambda x: x[len(x)-6:])
df_temp_melt['month'] = pd.to_datetime(df_temp_melt['month'], format='%b-%y')

# Sort time wise by location
df_temp_melt.sort_values(by=['patient_zip3', 'patient_state', 'month'], inplace=True)

# Fill missingness - forward, then backwards for remaining
df_temp_melt['value'] = df_temp_melt.groupby(['patient_zip3', 'patient_state'])['value'].ffill()
df_temp_melt['value'] = df_temp_melt.groupby(['patient_zip3', 'patient_state'])['value'].bfill()
df_temp_melt.head()

,patient_zip3,patient_state,variable,value,month
0,100,NY,Average of Jan-13,33.44,2013-01-01
763,100,NY,Average of Feb-13,30.84,2013-02-01
1526,100,NY,Average of Mar-13,38.45,2013-03-01
2289,100,NY,Average of Apr-13,51.26,2013-04-01
3052,100,NY,Average of May-13,60.18,2013-05-01


In [ ]:
# Reshape data to bring it in the initial format
df_temp_final = df_temp_melt.drop('month', axis=1).pivot(index=['patient_zip3', 'patient_state'],columns='variable', values='value').reset_index()[['patient_zip3', 'patient_state'] + temp_cols]
print(df_temp_final.shape)
df_temp_final.head()

(763, 74)


variable,patient_zip3,patient_state,Average of Jan-13,Average of Feb-13,Average of Mar-13,Average of Apr-13,Average of May-13,Average of Jun-13,Average of Jul-13,Average of Aug-13,Average of Sep-13,Average of Oct-13,Average of Nov-13,Average of Dec-13,Average of Jan-14,Average of Feb-14,Average of Mar-14,Average of Apr-14,Average of May-14,Average of Jun-14,Average of Jul-14,Average of Aug-14,Average of Sep-14,Average of Oct-14,Average of Nov-14,Average of Dec-14,Average of Jan-15,Average of Feb-15,Average of Mar-15,Average of Apr-15,Average of May-15,Average of Jun-15,Average of Jul-15,Average of Aug-15,Average of Sep-15,Average of Oct-15,Average of Nov-15,Average of Dec-15,Average of Jan-16,Average of Feb-16,Average of Mar-16,Average of Apr-16,Average of May-16,Average of Jun-16,Average of Jul-16,Average of Aug-16,Average of Sep-16,Average of Oct-16,Average of Nov-16,Average of Dec-16,Average of Jan-17,Average of Feb-17,Average of Mar-17,Average of Apr-17,Average of May-17,Average of Jun-17,Average of Jul-17,Average of Aug-17,Average of Sep-17,Average of Oct-17,Average of Nov-17,Average of Dec-17,Average of Jan-18,Average of Feb-18,Average of Mar-18,Average of Apr-18,Average of May-18,Average of Jun-18,Average of Jul-18,Average of Aug-18,Average of Sep-18,Average of Oct-18,Average of Nov-18,Average of Dec-18
0,100,NY,33.44,30.84,38.45,51.26,60.18,71.91,78.55,71.32,64.49,56.13,41.99,34.79,24.37,26.97,33.92,49.99,61.76,70.54,74.73,71.48,66.73,55.69,41.66,38.25,26.53,20.84,34.33,50.51,66.72,69.80,75.96,75.25,71.07,54.58,48.87,46.33,30.06,35.06,46.52,51.23,60.48,72.47,77.11,77.54,70.08,56.80,45.79,35.42,35.37,37.89,37.02,56.56,59.65,71.17,76.11,72.17,69.26,60.80,42.89,32.62,27.67,37.57,37.87,47.55,65.06,70.23,77.12,77.72,69.82,55.79,42.04,37.18
1,101,NY,33.51,30.81,38.44,51.27,60.14,71.98,78.64,71.34,64.53,56.15,42.03,34.93,24.42,26.97,33.92,33.92,61.76,61.76,74.75,71.51,66.77,55.65,41.72,38.32,26.62,20.97,34.39,50.56,66.82,69.84,76.02,75.35,71.20,54.72,48.96,46.43,30.09,35.16,46.64,51.34,60.55,72.67,77.19,77.67,70.22,56.95,45.83,35.54,35.45,37.99,37.09,56.75,59.79,71.37,76.24,72.29,69.38,60.93,43.01,32.70,27.77,37.69,37.95,47.68,65.17,70.33,77.22,77.83,69.93,55.92,42.16,37.26
2,103,NY,34.01,31.87,38.52,51.63,60.29,71.92,78.43,71.46,64.39,56.21,42.56,35.83,25.57,26.97,34.67,51.46,61.74,70.47,74.70,71.37,66.91,56.54,41.69,38.68,27.28,21.76,35.17,50.75,66.20,69.40,75.87,75.20,70.58,54.21,48.93,47.42,30.88,35.79,46.99,51.55,60.28,71.85,77.05,77.30,70.17,56.61,46.67,36.33,36.01,39.22,37.09,56.44,59.80,70.75,76.06,72.15,69.15,60.29,43.77,32.91,28.04,38.68,38.14,47.99,65.17,70.81,76.90,77.75,69.90,56.45,42.55,37.49
3,104,NY,32.81,31.04,38.55,51.15,60.48,71.39,77.86,71.19,64.10,55.98,41.73,33.68,23.95,26.97,33.92,49.99,61.76,70.54,74.55,71.22,66.44,56.05,41.21,37.70,25.83,19.78,33.79,50.08,65.92,69.54,75.48,74.45,70.01,53.43,48.15,45.55,29.80,34.24,45.55,50.29,59.92,70.84,76.48,76.47,68.93,55.55,45.48,34.50,34.72,37.03,36.50,55.00,58.52,69.56,75.05,71.20,68.33,59.81,41.91,31.91,26.81,36.58,37.22,46.54,64.18,69.41,76.37,76.81,68.91,54.75,41.06,36.54
4,105,NY,29.13,27.78,34.54,48.12,58.59,67.86,75.06,68.19,61.44,53.31,38.38,31.92,20.03,20.35,28.98,46.86,59.60,67.73,72.00,68.59,64.27,54.66,38.04,34.64,22.47,15.58,30.56,47.75,63.83,66.95,72.20,71.25,67.61,50.64,45.29,42.99,26.84,31.00,42.83,47.79,57.43,67.18,73.73,73.14,66.10,52.98,43.58,32.09,31.65,34.71,33.61,52.17,56.50,66.72,71.75,68.49,65.46,57.39,40.07,28.83,23.77,33.59,34.47,43.69,62.71,66.81,73.70,73.99,66.41,52.29,38.51,33.56


In [ ]:
df['payer_type'] = df['payer_type'].fillna('COMMERCIAL')

In [ ]:
df['patient_race'] = df['patient_race'].fillna('Missing')

In [ ]:
df_full = df.drop(pop_cols + temp_cols, axis=1).merge(
    df_pop, how='left', on=['patient_zip3', 'patient_state']
).merge(
    df_temp_final, how='left', on=['patient_zip3', 'patient_state']
)

# Also split back train and test
train = df_full[df_full.source == 'train'].drop('source', axis=1)
test = df_full[df_full.source == 'test'].drop('source', axis=1)

print(df_full.shape, df.shape, train.shape, test.shape)
df_full.head()

(18819, 149) (18819, 149) (13173, 148) (5646, 148)


,patient_id,patient_race,payer_type,patient_state,patient_zip3,Region,Division,patient_age,breast_cancer_diagnosis_code,breast_cancer_diagnosis_desc,metastatic_cancer_diagnosis_code,metastatic_diagnosis_period,source,population,density,age_median,age_under_10,age_10_to_19,age_20s,age_30s,age_40s,age_50s,age_60s,age_70s,age_over_80,male,female,married,divorced,never_married,widowed,family_size,family_dual_income,income_household_median,income_household_under_5,income_household_5_to_10,income_household_10_to_15,income_household_15_to_20,income_household_20_to_25,income_household_25_to_35,income_household_35_to_50,income_household_50_to_75,income_household_75_to_100,income_household_100_to_150,income_household_150_over,income_household_six_figure,income_individual_median,home_ownership,housing_units,home_value,rent_median,rent_burden,education_less_highschool,education_highschool,education_some_college,education_bachelors,education_graduate,education_college_or_above,education_stem_degree,labor_force_participation,unemployment_rate,self_employed,farmer,race_white,race_black,race_asian,race_native,race_pacific,race_other,race_multiple,hispanic,disabled,poverty,limited_english,commute_time,health_uninsured,veteran,Average of Jan-13,Average of Feb-13,Average of Mar-13,Average of Apr-13,Average of May-13,Average of Jun-13,Average of Jul-13,Average of Aug-13,Average of Sep-13,Average of Oct-13,Average of Nov-13,Average of Dec-13,Average of Jan-14,Average of Feb-14,Average of Mar-14,Average of Apr-14,Average of May-14,Average of Jun-14,Average of Jul-14,Average of Aug-14,Average of Sep-14,Average of Oct-14,Average of Nov-14,Average of Dec-14,Average of Jan-15,Average of Feb-15,Average of Mar-15,Average of Apr-15,Average of May-15,Average of Jun-15,Average of Jul-15,Average of Aug-15,Average of Sep-15,Average of Oct-15,Average of Nov-15,Average of Dec-15,Average of Jan-16,Average of Feb-16,Average of Mar-16,Average of Apr-16,Average of May-16,Average of Jun-16,Average of Jul-16,Average of Aug-16,Average of Sep-16,Average of Oct-16,Average of Nov-16,Average of Dec-16,Average of Jan-17,Average of Feb-17,Average of Mar-17,Average of Apr-17,Average of May-17,Average of Jun-17,Average of Jul-17,Average of Aug-17,Average of Sep-17,Average of Oct-17,Average of Nov-17,Average of Dec-17,Average of Jan-18,Average of Feb-18,Average of Mar-18,Average of Apr-18,Average of May-18,Average of Jun-18,Average of Jul-18,Average of Aug-18,Average of Sep-18,Average of Oct-18,Average of Nov-18,Average of Dec-18
0,268700,Missing,COMMERCIAL,AR,724,South,AR,39,C50912,Malignant neoplasm of unspecified site of left female breast,C773,191,train,3924.87,82.63,42.58,11.61,13.03,10.87,11.80,12.29,13.22,13.47,10.07,3.64,51.43,48.57,51.05,16.72,23.57,8.66,3.01,43.99,44483.35,2.21,3.97,8.52,7.08,7.67,13.82,15.14,17.51,11.26,8.90,3.93,12.83,24048.55,72.11,1513.75,87384.33,641.39,27.52,16.55,41.83,28.31,9.21,4.11,13.32,38.78,53.60,5.85,11.82,5.31,92.95,1.73,0.33,0.20,0.03,0.83,3.94,3.03,22.24,19.27,0.42,25.35,8.06,8.11,38.55,39.88,42.75,55.16,65.17,75.98,76.75,76.45,73.67,59.73,45.18,37.43,31.67,33.83,42.35,57.72,67.35,75.92,74.28,79.59,70.84,62.39,41.89,41.46,35.24,31.10,45.50,60.94,68.49,77.69,81.35,76.28,73.32,60.32,51.00,48.37,35.76,42.89,53.21,61.07,66.29,82.78,81.52,79.52,75.61,65.76,53.85,39.56,41.59,49.01,52.28,63.40,67.11,75.78,80.58,76.16,72.91,62.68,49.73,39.80,31.71,41.69,50.22,52.55,74.77,79.96,81.69,78.30,74.56,59.98,42.98,41.18
1,484983,White,COMMERCIAL,IL,629,Midwest,IL,55,C50412,Malig neoplasm of upper-outer quadrant of left female breast,C773,33,train,2745.39,51.79,43.54,11.22,12.19,11.45,11.01,11.35,14.39,14.15,9.17,5.05,49.32,50.68,49.48,15.42,26.93,8.17,3.17,41.41,51796.79,3.67,3.86,6.58,5.58,5.38,11.02,13.09,19.56,11.76,11.40,8.11,19.51,28028.04,76.71,1113.35,92026.84,638.60,29.37,10.93,35.26,35.33,12.46,6.04,18.49,36.35,52.51,7.45,9.19,5.21,88.75,6.44,0.53,0.19,0.05,0.61,3.42,2.78,20.16,16.94,0.43,26.26,6.93,9.71,34.85,36.15,39.41,54.63,65.41,73.89,

In [ ]:
df.isnull().sum()

patient_id                            0
patient_race                          0
payer_type                            0
patient_state                         0
patient_zip3                          0
Region                                0
Division                              0
patient_age                           0
breast_cancer_diagnosis_code          0
breast_cancer_diagnosis_desc          0
metastatic_cancer_diagnosis_code      0
population                            0
density                               0
age_median                            0
age_under_10                          0
age_10_to_19                          0
age_20s                               0
age_30s                               0
age_40s                               0
age_50s                               0
age_60s                               0
age_70s                               0
age_over_80                           0
male                                  0
female                                0


In [ ]:
months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
# Iterating from each month to get average temperature in train as well as test data
for i in months:
    cols = [col for col in train.columns if col.startswith(f'Average of {i}')]
    train[f'Average of {i}'] = train[cols].sum(axis = 1)/len(cols)
    test[f'Average of {i}'] =test[cols].sum(axis = 1)/len(cols)
    train.drop(cols,axis = 1, inplace = True) # droping those columns whose average are taken above
    test.drop(cols,axis =1, inplace = True)

print(test.columns)
print(train.columns)

Index(['patient_id', 'patient_race', 'payer_type', 'patient_state',
       'patient_zip3', 'Region', 'Division', 'patient_age',
       'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc',
       'metastatic_cancer_diagnosis_code', 'metastatic_diagnosis_period',
       'population', 'density', 'age_median', 'age_under_10', 'age_10_to_19',
       'age_20s', 'age_30s', 'age_40s', 'age_50s', 'age_60s', 'age_70s',
       'age_over_80', 'male', 'female', 'married', 'divorced', 'never_married',
       'widowed', 'family_size', 'family_dual_income',
       'income_household_median', 'income_household_under_5',
       'income_household_5_to_10', 'income_household_10_to_15',
       'income_household_15_to_20', 'income_household_20_to_25',
       'income_household_25_to_35', 'income_household_35_to_50',
       'income_household_50_to_75', 'income_household_75_to_100',
       'income_household_100_to_150', 'income_household_150_over',
       'income_household_six_figure', 'income_indivi

In [ ]:
cols = train.columns[train.dtypes == 'object']
cols

Index(['patient_race', 'payer_type', 'patient_state', 'Region', 'Division',
       'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc',
       'metastatic_cancer_diagnosis_code'],
      dtype='object')

In [ ]:
# we wont be using disgnosis desc as it was used to clean the data already not requried anymore so we will drop it
train.drop(['breast_cancer_diagnosis_desc'],axis = 1 ,inplace = True)
test.drop(['breast_cancer_diagnosis_desc'],axis = 1 ,inplace = True)

In [ ]:
!pip install category_encoders

In [ ]:
from category_encoders import TargetEncoder
cols = ['patient_race','payer_type', 'patient_state', 'Region', 'Division', 'breast_cancer_diagnosis_code', 'metastatic_cancer_diagnosis_code']

combined_df = pd.concat([train, test], axis=0)
# Target Encoder
TE = TargetEncoder()

TE = TargetEncoder()
combined_df[cols] = TE.fit_transform(combined_df[cols],combined_df['metastatic_diagnosis_period'])

In [ ]:
train_target_encoded = combined_df.iloc[:len(train)]
test_target_encoded = combined_df.iloc[len(train):]



In [ ]:
import pandas as pd

# Assuming train_target_encoded and test_target_encoded are your DataFrames containing income-related columns
# Replace train_target_encoded and test_target_encoded with your actual DataFrame names

# Define the income range categories and corresponding column names
income_ranges = {
    '5k_to_15k': ['income_household_5_to_10', 'income_household_10_to_15'],
    '15k_to_25k': ['income_household_15_to_20', 'income_household_20_to_25'],
    '25k_to_35k': ['income_household_25_to_35'],
    '35k_to_75k': ['income_household_35_to_50', 'income_household_50_to_75'],
    '75k_to_150k': ['income_household_75_to_100', 'income_household_100_to_150']
}

# Reduce dimensionality for train_target_encoded
train_target_encoded['5k_to_15k'] = train_target_encoded[income_ranges['5k_to_15k']].sum(axis=1)
train_target_encoded['15k_to_25k'] = train_target_encoded[income_ranges['15k_to_25k']].sum(axis=1)
train_target_encoded['25k_to_35k'] = train_target_encoded[income_ranges['25k_to_35k']]
train_target_encoded['35k_to_75k'] = train_target_encoded[income_ranges['35k_to_75k']].sum(axis=1)
train_target_encoded['75k_to_150k'] = train_target_encoded[income_ranges['75k_to_150k']].sum(axis=1)

# Drop the original columns
train_target_encoded = train_target_encoded.drop(columns=[col for cols in income_ranges.values() for col in cols])

# Reduce dimensionality for test_target_encoded
test_target_encoded['5k_to_15k'] = test_target_encoded[income_ranges['5k_to_15k']].sum(axis=1)
test_target_encoded['15k_to_25k'] = test_target_encoded[income_ranges['15k_to_25k']].sum(axis=1)
test_target_encoded['25k_to_35k'] = test_target_encoded[income_ranges['25k_to_35k']]
test_target_encoded['35k_to_75k'] = test_target_encoded[income_ranges['35k_to_75k']].sum(axis=1)
test_target_encoded['75k_to_150k'] = test_target_encoded[income_ranges['75k_to_150k']].sum(axis=1)

# Drop the original columns
test_target_encoded = test_target_encoded.drop(columns=[col for cols in income_ranges.values() for col in cols])

# Now train_target_encoded and test_target_encoded contain reduced dimensionality income-related columns



In [ ]:
test_target_encoded.isnull().sum()



patient_id                          0
patient_race                        0
payer_type                          0
patient_state                       0
patient_zip3                        0
Region                              0
Division                            0
patient_age                         0
breast_cancer_diagnosis_code        0
metastatic_cancer_diagnosis_code    0
metastatic_diagnosis_period         0
population                          0
density                             0
age_median                          0
age_under_10                        0
age_10_to_19                        0
age_20s                             0
age_30s                             0
age_40s                             0
age_50s                             0
age_60s                             0
age_70s                             0
age_over_80                         0
male                                0
female                              0
married                             0
divorced    

In [ ]:
# Checking shape after encoding
print("Shape of encoded_train:", train_target_encoded.shape)
print("Shape of encoded_test:", test_target_encoded.shape)

Shape of encoded_train: (13173, 87)
Shape of encoded_test: (5646, 87)


In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV
# Assuming 'combined_df' and 'train' are already defined
# Create 'train_target_encoded' from the combined_df
train_target_encoded = combined_df.iloc[:len(train)]

# Replace 'target_column' with the actual name of your target variable column
target_column = 'metastatic_diagnosis_period'  # Change this to your target column name

# Split the features and target variable
X = train_target_encoded.drop(columns=[target_column])
y = train_target_encoded[target_column]

# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=['object', 'category']).columns
numerical_cols = X.select_dtypes(include=['number']).columns

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('chi2', SelectKBest(chi2, k='all')),  # You can adjust 'k' to select top features
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Preprocessing for numerical data
numerical_transformer = StandardScaler()

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.isnull().sum()


patient_id                          0
patient_race                        0
payer_type                          0
patient_state                       0
patient_zip3                        0
Region                              0
Division                            0
patient_age                         0
breast_cancer_diagnosis_code        0
metastatic_cancer_diagnosis_code    0
population                          0
density                             0
age_median                          0
age_under_10                        0
age_10_to_19                        0
age_20s                             0
age_30s                             0
age_40s                             0
age_50s                             0
age_60s                             0
age_70s                             0
age_over_80                         0
male                                0
female                              0
married                             0
divorced                            0
never_marrie

In [ ]:
rf = RandomForestRegressor(random_state=42)

# Train the model
rf.fit(X_train, y_train)

# Predict on the test set
y_pred = rf.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error: {rmse}")

Root Mean Squared Error: 85.71462968927999
